In [1]:
# Good old pandas and numpy
import pandas as pd
import numpy as np

# Unfortunately I'm still using matplotlib for graphs
import matplotlib.pyplot as plt
import seaborn as sns

import argparse
import logging
import os

import numpy as np
import torch
import torch.optim as optim
import utils
import model.net as net
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from torch.utils.data.sampler import RandomSampler
from timeit import default_timer as timer
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, STATUS_FAIL
from tqdm import tqdm
from torch import nn
from evaluate import evaluate
from dataloader import *
from pytorchtools import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from hyperopt import hp

logger = logging.getLogger('DeepAR.Train')
init_params = utils.Params('../init_params.json')

In [2]:
torch.__version__

'1.5.1+cu92'

In [3]:
def create_directory(save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

In [4]:
def train(model: nn.Module,
          optimizer: optim,
          loss_fn,
          train_loader: DataLoader,
          test_loader: DataLoader,
          params: utils.Params,
          epoch: int) -> float:
    '''Train the model on one epoch by batches.
    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes outputs and labels per timestep, and then computes the loss for the batch
        train_loader: load train data and labels
        test_loader: load test data and labels
        params: (Params) hyperparameters
        epoch: (int) the current training epoch
    '''
    model.train()
    loss_epoch = np.zeros(len(train_loader))
    # Train_loader:
    # train_batch ([batch_size, train_window, 1+cov_dim]): z_{0:T-1} + x_{1:T}, note that z_0 = 0;
    # idx ([batch_size]): one integer denoting the time series id;
    # labels_batch ([batch_size, train_window]): z_{1:T}.
    for i, (train_batch, idx, labels_batch) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        batch_size = train_batch.shape[0]

        train_batch = train_batch.permute(1, 0, 2).to(torch.float32).to(params.device)  # not scaled
        labels_batch = labels_batch.permute(1, 0).to(torch.float32).to(params.device)  # not scaled
        idx = idx.unsqueeze(0).to(params.device)

        loss = torch.zeros(1, device=params.device)
        hidden = model.init_hidden(batch_size)
        cell = model.init_cell(batch_size)

        for t in range(params.train_window):
            # if z_t is missing, replace it by output mu from the last time step
            zero_index = (train_batch[t, :, 0] == 0)
            if t > 0 and torch.sum(zero_index) > 0:
                train_batch[t, zero_index, 0] = mu[zero_index]
            mu, sigma, hidden, cell = model(train_batch[t].unsqueeze_(0).clone(), idx, hidden, cell)
            loss += loss_fn(mu, sigma, labels_batch[t])

        loss.backward()
        optimizer.step()
        loss = loss.item() / params.train_window  # loss per timestep
        loss_epoch[i] = loss
        if i % 1000 == 0:
            #print('len(test_loader): ',len(test_loader))
            test_metrics = evaluate(model, loss_fn, test_loader, params, epoch, sample=args.sampling, plot_graph=True)
            model.train()
            logger.info(f'train_loss: {loss}')
    return loss_epoch


def train_and_evaluate(model: nn.Module,
                       train_loader: DataLoader,
                       test_loader: DataLoader,
                       optimizer: optim, loss_fn,
                       params: utils.Params,
                       restore_file: str = None) -> None:
    '''Train the model and evaluate every epoch.
    Args:
        model: (torch.nn.Module) the Deep AR model
        train_loader: load train data and labels
        test_loader: load test data and labels
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes outputs and labels per timestep, and then computes the loss for the batch
        params: (Params) hyperparameters
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    '''
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(params.model_dir, restore_file + '.pth.tar')
        logger.info('Restoring parameters from {}'.format(restore_path))
        utils.load_checkpoint(restore_path, model, optimizer)
    logger.info('begin training and evaluation')
    
    global global_best_test_ND
    logger.info('Global best: {}'.format(global_best_test_ND))
    best_test_ND = float('inf') 
    
    train_len = len(train_loader)
    ND_summary = np.zeros(params.num_epochs)
    loss_summary = np.zeros((train_len * params.num_epochs))
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=10, verbose=True)
    
    for epoch in range(params.num_epochs):
        logger.info('Epoch {}/{}'.format(epoch + 1, params.num_epochs))
        loss_summary[epoch * train_len:(epoch + 1) * train_len] = train(model, optimizer, loss_fn, train_loader,
                                                                        test_loader, params, epoch)
        test_metrics = evaluate(model, loss_fn, test_loader, params, epoch, sample=args.sampling)
        
        ND_summary[epoch] = test_metrics['ND']
        is_best = ND_summary[epoch] <= best_test_ND
        
        if is_best:
            logger.info('- Found new best ND')
            best_test_ND = ND_summary[epoch]
            is_global_best = best_test_ND <= global_best_test_ND  
            
            if is_global_best:
                global_best_test_ND = best_test_ND
                # Save weights
                utils.save_checkpoint({'epoch': epoch + 1,
                                       'state_dict': model.state_dict(),
                                       'optim_dict': optimizer.state_dict()},
                                      epoch=epoch,
                                      is_best=is_global_best,
                                      checkpoint=params.model_dir,
                                      dataset = init_params.dataset)
                
                best_json_path = os.path.join(params.model_dir, 'metrics_test_best_weights.json')
                utils.save_dict_to_json(test_metrics, best_json_path)

        logger.info('Current Best ND is: %.5f' % best_test_ND)

        utils.plot_all_epoch(ND_summary[:epoch + 1], args.dataset + '_ND', params.plot_dir)
        utils.plot_all_epoch(loss_summary[:(epoch + 1) * train_len], args.dataset + '_loss', params.plot_dir)

        last_json_path = os.path.join(params.model_dir, 'metrics_test_last_weights.json')
        utils.save_dict_to_json(test_metrics, last_json_path)
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(test_metrics['test_loss'], model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break

    if args.save_best:
        f = open('./param_search.txt', 'w')
        f.write('-----------\n')
        list_of_params = args.search_params.split(',')
        print_params = ''
        for param in list_of_params:
            param_value = getattr(params, param)
            print_params += f'{param}: {param_value:.2f}'
        print_params = print_params[:-1]
        f.write(print_params + '\n')
        f.write('Best ND: ' + str(best_test_ND) + '\n')
        logger.info(print_params)
        logger.info(f'Best ND: {best_test_ND}')
        f.close()
        utils.plot_all_epoch(ND_summary, print_params + '_ND', location=params.plot_dir)
        utils.plot_all_epoch(loss_summary, print_params + '_loss', location=params.plot_dir)
        
    return best_test_ND

In [5]:

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default=init_params.dataset , help='Name of the dataset')
parser.add_argument('--data-folder', default='data', help='Parent dir of the dataset')
parser.add_argument('--model-name', default='base_model', help='Directory containing params.json')
parser.add_argument('--relative-metrics', action='store_true', help='Whether to normalize the metrics by label scales')
parser.add_argument('--sampling', action='store_true', help='Whether to sample during evaluation')
parser.add_argument('--save-best', action='store_true', help='Whether to save best ND to param_search.txt')
parser.add_argument('--restore-file', default='None',
                    help='Optional, name of the file in --model_dir containing weights to reload before \
                    training')  # 'best' or 'epoch_#'


_StoreAction(option_strings=['--restore-file'], dest='restore_file', nargs=None, const=None, default='None', type=None, choices=None, help='Optional, name of the file in --model_dir containing weights to reload before                     training', metavar=None)

In [6]:
class Args:
    dataset = init_params.dataset
    data_folder ='data'
    model_name ='base_model'
    relative_metrics = False
    sampling = False
    save_best = False
    restore_file = None

In [7]:
#if __name__ == '__main__':

# Load the parameters from json file
args = Args()
model_dir = os.path.join('experiments', args.model_name)
json_path = os.path.join(model_dir, 'params.json')
data_dir = os.path.join(args.data_folder, args.dataset)
assert os.path.isfile(json_path), f'No json configuration file found at {json_path}'
params = utils.Params(json_path)

params.relative_metrics = args.relative_metrics
params.sampling =  args.sampling
params.model_dir = model_dir
params.plot_dir = os.path.join(model_dir, 'figures')

utils.set_logger(os.path.join(model_dir, 'train_optimization.log'))
logger.info('Loading the datasets...')

train_set = TrainDataset(data_dir, args.dataset, params.num_class)
test_set = TestDataset(data_dir, args.dataset, params.num_class)
sampler = WeightedSampler(data_dir, args.dataset) # Use weighted sampler instead of random sampler
train_loader = DataLoader(train_set, batch_size=params.batch_size, sampler=sampler, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=params.predict_batch, sampler=RandomSampler(test_set), num_workers=4, pin_memory=True)
logger.info('Loading complete.')


# create missing directories
try:
    os.mkdir(params.plot_dir)
except FileExistsError:
    pass

# use GPU if available
cuda_exist = torch.cuda.is_available()
# Set random seeds for reproducible experiments if necessary
if cuda_exist:
    params.device = torch.device('cuda')
    # torch.cuda.manual_seed(240)
    logger.info('Using Cuda...')
    model = net.Net(params).cuda()
    print('Using Cuda')
else:
    params.device = torch.device('cpu')
    # torch.manual_seed(230)
    logger.info('Not using cuda...')
    model = net.Net(params)
    print('Not Using Cuda')

logger.info(f'Model: \n{str(model)}')
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)

# fetch loss function
loss_fn = net.loss_fn

# Train the model
logger.info('Starting training for {} epoch(s)'.format(params.num_epochs))

[10:38:50] DeepAR.Train: Loading the datasets...
[10:38:50] DeepAR.Data: train_len: 1364
[10:38:50] DeepAR.Data: building datasets from data\Dhaka...
[10:38:50] DeepAR.Data: test_len: 359
[10:38:50] DeepAR.Data: building datasets from data\Dhaka...
[10:38:50] DeepAR.Data: weights: tensor([0.0014, 0.0014, 0.0014,  ..., 0.0009, 0.0009, 0.0009],
       dtype=torch.float64)
[10:38:50] DeepAR.Data: num samples: 1364
[10:38:50] DeepAR.Train: Loading complete.
[10:38:50] DeepAR.Train: Using Cuda...
Using Cuda
[10:38:52] DeepAR.Train: Model: 
Net(
  (embedding): Embedding(1, 30)
  (lstm): LSTM(47, 40, num_layers=2, dropout=0.1)
  (relu): ReLU()
  (distribution_mu): Linear(in_features=80, out_features=1, bias=True)
  (distribution_presigma): Linear(in_features=80, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1, threshold=20)
)
[10:38:52] DeepAR.Train: Starting training for 2 epoch(s)


There is a number of different domain distribution types:
choice : categorical variables
'class_weight': hp.choice('class_weight', [None, 'balanced']),
quniform : discrete uniform (integers spaced evenly)
'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
uniform: continuous uniform (floats spaced evenly)
'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
loguniform: continuous log uniform (floats spaced evenly on a log scale)
'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),

In [8]:
# start = timer()
# train_and_evaluate(model, train_loader, test_loader, optimizer, loss_fn, params, args.restore_file)
# end = timer()
# # Calculate time to evaluate
# print('Time Taken: ', end - start)

#### Optimization

In [9]:
# Define the search space
space = {
    'lstm_layers': hp.choice('lstm_layers', np.arange(1, 5, 1, dtype=int)),
    'batch_size': hp.choice('batch_size', np.arange(8, 64, 8, dtype=int)),
    'lstm_hidden_dim': hp.choice('lstm_hidden_dim', np.arange(30, 200, 10, dtype=int)),
    'embedding_dim': hp.choice('embedding_dim', np.arange(1, 60, 10, dtype=int)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2))
}

global_best_test_ND = float('inf')

In [10]:
def objective(param_space):
    """Objective function to minimize with smarter return values"""
    
    global global_best_test_ND 
    params = utils.Params(json_path)
    
    # Evaluate the function
    start = timer()

    params.lstm_layers = param_space['lstm_layers']
    params.lstm_hidden_dim = param_space['lstm_hidden_dim']
    params.embedding_dim = param_space['embedding_dim']
    params.learning_rate = param_space['learning_rate']
    params.batch_size = param_space['batch_size']
    
    if cuda_exist:
        params.device = torch.device('cuda')
    else:
        params.device = torch.device('cpu')

    params.relative_metrics = args.relative_metrics
    params.sampling =  args.sampling
    params.model_dir = model_dir
    params.plot_dir = os.path.join(model_dir, 'figures')
    
    optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    loss = train_and_evaluate(model, train_loader, test_loader, optimizer, loss_fn, params, args.restore_file)
    end = timer()
    
    # Calculate time to evaluate
    time_elapsed = end - start
    results = {'loss': loss, 'status': STATUS_OK, \
               'lstm_layers': param_space['lstm_layers'], \
               'lstm_hidden_dim': param_space['lstm_hidden_dim'], \
               'embedding_dim': param_space['embedding_dim'], \
               'learning_rate': param_space['learning_rate'], \
               'batch_size': param_space['batch_size'], \
               'time': time_elapsed}

    # Return dictionary
    return results

start_optimization = timer()

# Create the algorithm
tpe_algo = tpe.suggest

# New trials object
trials = Trials()

# Run 50 evals with the tpe algorithm
best = fmin(fn=objective, space=space, algo=tpe_algo, trials=trials, 
                max_evals=init_params.max_number_of_trials, rstate= np.random.RandomState(120))
end_optimization = timer()
print('\nTime Elapsed(sec): ' , end_optimization - start_optimization)

[10:38:52] DeepAR.Train: begin training and evaluation                                                                 
[10:38:52] DeepAR.Train: Global best: inf                                                                              
[10:38:52] DeepAR.Train: Epoch 1/2                                                                                     
  0%|                                                                            | 0/3 [00:00<?, ?trial/s, best loss=?]

  0%|                                                                                           | 0/43 [00:00<?, ?it/s]

  0%|                                                                                          | 0/359 [00:00<?, ?it/s]

  0%|2                                                                                 | 1/359 [00:04<25:42,  4.31s/it]

  1%|4                                                                                 | 2/359 [00:04<18:16,  3.07s/it]

  1%|6                                                                                 | 3/359 [00:04<13:02,  2.20s/it]

  1%|9                                                                                 | 4/359 [00:04<09:26,  1.59s/it]

  1%|#1                                                                                | 5/359 [00:05<06:52,  1.17s/it]

  2%|#3                                                                                | 6/359 [00:05<05:05,  1.16it/s]

  2%|#5                         


 18%|##############4                                                                  | 64/359 [00:15<00:58,  5.02it/s]

 18%|##############6                                                                  | 65/359 [00:15<01:00,  4.84it/s]

 18%|##############8                                                                  | 66/359 [00:16<01:02,  4.71it/s]

 19%|###############1                                                                 | 67/359 [00:16<01:03,  4.60it/s]

 19%|###############3                                                                 | 68/359 [00:16<01:03,  4.61it/s]

 19%|###############5                                                                 | 69/359 [00:16<01:02,  4.62it/s]

 19%|###############7                                                                 | 70/359 [00:16<01:02,  4.61it/s]

 20%|################                                                                 | 71/359 [00:17<01:01,  4.68it/s]

 20%|################2         


 36%|############################7                                                   | 129/359 [00:27<00:38,  5.99it/s]

 36%|############################9                                                   | 130/359 [00:27<00:37,  6.14it/s]

 36%|#############################1                                                  | 131/359 [00:28<00:36,  6.18it/s]

 37%|#############################4                                                  | 132/359 [00:28<00:37,  6.01it/s]

 37%|#############################6                                                  | 133/359 [00:28<00:38,  5.86it/s]

 37%|#############################8                                                  | 134/359 [00:28<00:39,  5.70it/s]

 38%|##############################                                                  | 135/359 [00:28<00:38,  5.83it/s]

 38%|##############################3                                                 | 136/359 [00:28<00:37,  5.96it/s]

 38%|##########################


 54%|###########################################2                                    | 194/359 [00:38<00:26,  6.12it/s]

 54%|###########################################4                                    | 195/359 [00:38<00:26,  6.12it/s]

 55%|###########################################6                                    | 196/359 [00:38<00:26,  6.14it/s]

 55%|###########################################8                                    | 197/359 [00:39<00:26,  6.10it/s]

 55%|############################################1                                   | 198/359 [00:39<00:26,  6.05it/s]

 55%|############################################3                                   | 199/359 [00:39<00:27,  5.86it/s]

 56%|############################################5                                   | 200/359 [00:39<00:26,  5.90it/s]

 56%|############################################7                                   | 201/359 [00:39<00:26,  5.87it/s]

 56%|##########################

...


 43%|##################################7                                             | 156/359 [00:31<00:34,  5.93it/s]

 44%|##################################9                                             | 157/359 [00:32<00:34,  5.90it/s]

 44%|###################################2                                            | 158/359 [00:32<00:33,  5.96it/s]

 44%|###################################4                                            | 159/359 [00:32<00:33,  5.92it/s]

 45%|###################################6                                            | 160/359 [00:32<00:34,  5.82it/s]

 45%|###################################8                                            | 161/359 [00:32<00:33,  5.94it/s]

 45%|####################################1                                           | 162/359 [00:32<00:32,  6.00it/s]

 45%|####################################3                                           | 163/359 [00:33<00:33,  5.87it/s]

 46%|##########################


 62%|#################################################6                              | 223/359 [00:43<00:24,  5.59it/s]

 62%|#################################################9                              | 224/359 [00:43<00:23,  5.75it/s]

 63%|##################################################1                             | 225/359 [00:43<00:23,  5.64it/s]

 63%|##################################################3                             | 226/359 [00:44<00:24,  5.39it/s]

 63%|##################################################5                             | 227/359 [00:44<00:24,  5.38it/s]

 64%|##################################################8                             | 228/359 [00:44<00:24,  5.35it/s]

 64%|###################################################                             | 229/359 [00:44<00:24,  5.36it/s]

 64%|###################################################2                            | 230/359 [00:44<00:24,  5.30it/s]

 64%|##########################


 81%|################################################################6               | 290/359 [00:55<00:12,  5.74it/s]

 81%|################################################################8               | 291/359 [00:55<00:11,  5.85it/s]

 81%|#################################################################               | 292/359 [00:56<00:11,  5.86it/s]

 82%|#################################################################2              | 293/359 [00:56<00:11,  5.81it/s]

 82%|#################################################################5              | 294/359 [00:56<00:11,  5.67it/s]

 82%|#################################################################7              | 295/359 [00:56<00:11,  5.81it/s]

 82%|#################################################################9              | 296/359 [00:56<00:10,  5.91it/s]

 83%|##################################################################1             | 297/359 [00:56<00:10,  5.82it/s]

 83%|##########################


 99%|###############################################################################5| 357/359 [01:09<00:00,  6.03it/s]

100%|###############################################################################7| 358/359 [01:09<00:00,  5.98it/s]

100%|################################################################################| 359/359 [01:09<00:00,  6.04it/s]

100%|################################################################################| 359/359 [01:09<00:00,  5.14it/s]


[10:56:48] DeepAR.Eval: - Full test metrics: ND: 0.459; RMSE: 0.584; test_loss: 4.748                                  
[10:56:48] DeepAR.Train: - Found new best ND                                                                           
[10:56:48] DeepAR.Train: Current Best ND is: 0.45878                                                                   
Validation loss decreased (6.812364 --> 4.747932).  Saving model ...                                                   
100%|████████████████████████████████████████████████| 3/3 [17:56<00:00, 358.84s/trial, best loss: 0.28093163413118427]
Time Elapsed(sec):  1076.5233607999999


In [11]:
# Results into a dataframe
results = trials.results
results_df = pd.DataFrame({'time': [x['time'] for x in results], 
                           'loss': [x['loss'] for x in results],
                           'lstm_layers': [x['lstm_layers'] for x in results],
                           'lstm_hidden_dim': [x['lstm_hidden_dim'] for x in results],
                           'embedding_dim': [x['embedding_dim'] for x in results],
                           'learning_rate': [x['learning_rate'] for x in results],
                           'batch_size': [x['batch_size'] for x in results],
                            'iteration': list(range(len(results)))})

# Sort with lowest loss on top
results_df = results_df.sort_values('loss', ascending = True)
results_df.to_csv('search_result_{}_{}_trials.csv'.format(init_params.dataset, init_params.max_number_of_trials))
results_df.head()

,time,loss,lstm_layers,lstm_hidden_dim,embedding_dim,learning_rate,batch_size,iteration
1,369.853527,0.280932,3,110,41,0.030934,24,1
2,349.485432,0.458777,4,160,41,0.093566,56,2
0,357.152484,0.459854,1,170,11,0.069555,32,0


In [12]:
results_df.to_csv('search_result_{}_trials.csv'.format(init_params.max_number_of_trials))

In [13]:
results_df.plot.scatter(x='iteration', y='loss', c='DarkBlue')
plt.savefig('search_result_{}_{}_trials.png'.format(init_params.dataset, init_params.max_number_of_trials))

#### Evaluate Prediction

In [14]:
### Evaluate prediction on test
#!python evaluate_test.py
%run evaluate_test

  0%|                                                                                          | 0/359 [00:00<?, ?it/s]

[04:07:51] DeepAR.Data: test_len: 359
[04:07:51] DeepAR.Data: test_len: 359
[04:07:51] DeepAR.Data: building datasets from data\Dhaka...
[04:07:51] DeepAR.Data: building datasets from data\Dhaka...
model:  Net(
  (embedding): Embedding(1, 30)
  (lstm): LSTM(47, 40, num_layers=2, dropout=0.1)
  (relu): ReLU()
  (distribution_mu): Linear(in_features=80, out_features=1, bias=True)
  (distribution_presigma): Linear(in_features=80, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1, threshold=20)
)


100%|████████████████████████████████████████████████████████████████████████████████| 359/359 [01:07<00:00,  5.35it/s]


In [15]:
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')

In [16]:
# Load Prediction
prediction = pd.read_csv(os.path.join(data_dir, 'DeepAR_prediction_test.csv'), converters={'prediction':converter, 'ground_truth':converter,\
                                                                 'history':converter})
prediction

,Unnamed: 0,Date,history,ground_truth,prediction
0,0,2017-06-23 02:00:00,"[71.300156, 85.68237, 58.422752, 92.77406, 88....","[35.058533, 33.11897, 33.638298, 35.980972, 37...","[29.23979, 29.237457, 29.236912, 29.236547, 29..."
1,1,2017-03-12 01:00:00,"[138.55583, 121.7033, 152.07426, 152.21964, 13...","[69.93416, 104.75256, 96.363235, 78.1666, 78.4...","[99.03088, 99.12319, 98.90429, 98.827034, 98.6..."
2,2,2017-12-04 01:00:00,"[33.904625, 31.085726, 36.093254, 23.10713, 18...","[151.73, 152.17035, 165.38461, 159.82382, 161....","[108.34194, 104.79248, 103.57372, 101.582794, ..."
3,3,2017-06-09 02:00:00,"[90.01698, 72.58378, 69.93416, 104.75256, 96.3...","[31.877853, 24.876614, 28.553947, 38.04301, 52...","[34.237522, 34.207092, 34.179108, 34.20502, 34..."
4,4,2017-01-23 01:00:00,"[61.65644, 54.610603, 43.395645, 38.970524, 57...","[145.8847, 143.78395, 158.23164, 156.89087, 17...","[173.33377, 176.77905, 175.87457, 177.33301, 1..."
...,...,...,...,...,...
354,354,2017-06-12 02:00:00,"[104.75256, 96.363235, 78.1666, 78.47042, 59.8...","[38.04301, 52.33765, 40.153736, 34.9688, 21.14...","[33.26722, 33.26665, 33.251072, 33.24057, 33.2..."
355,355,2017-10-04 02:00:00,"[33.533714, 33.666843, 33.799976, 33.933105, 2...","[44.122803, 28.539381, 41.379723, 45.47029, 52...","[55.2739, 51.42646, 49.25162, 46.81185, 49.307..."
356,356,2017-11-09 01:00:00,"[40.936817, 19.272875, 25.220804, 29.098726, 3...","[99.03619, 72.84431, 70.47379, 89.101974, 83.6...","[76.06438, 70.91708, 66.2915, 49.049374, 53.01..."
357,357,2017-01-09 01:00:00,"[82.76314, 35.45005, 61.921047, 63.19193, 41.6...","[200.39351, 123.46756, 173.84656, 172.80728, 1...","[171.46686, 160.41676, 152.74376, 144.27245, 1..."


In [17]:
def get_rmse_by_days(prediction):
    
    day_ground_truth_array = np.stack(prediction.ground_truth.values)
    day_prediction_array = np.stack(prediction.prediction.values)
    rmse_day = []
    for day in range(7):
        mse = mean_squared_error(day_ground_truth_array[:,day], day_prediction_array[:,day])
        #print(mse)
        print('Day:', day, sqrt(mse))
        rmse_day.append(sqrt(mse))
    print('\nMean RMSE: ', sum(rmse_day)/len(rmse_day))

def plot_prediction(x, y_true, y_pred, prediction_dates, model = 'lstm', train = False, plot = False):
    """Plots the predictions.
    
    Arguments
    ---------
    x: Input sequence of shape (input_sequence_length,
        dimension_of_signal)
    y_true: True output sequence of shape (input_sequence_length,
        dimension_of_signal)
    y_pred: Predicted output sequence (input_sequence_length,
        dimension_of_signal)
    """
    total_predctions = x.shape[0]
    rmse_list = []
    mae_list = []
    save_path = os.path.join('data', init_params.dataset, 'visualizations')
    create_directory(save_path)
    
    for j in range(total_predctions):
        past = x[j][:]
        true = y_true[j][:]
        pred = y_pred[j][:]
        rmse = sqrt(mean_squared_error(pred, true))
        mae = mean_absolute_error(pred, true)
        rmse_list.append(rmse)   
        mae_list.append(mae)
        label1 = "Seen (past) values" if j==0 else "_nolegend_"
        label2 = "True future values" if j==0 else "_nolegend_"
        label3 = "Predictions" if j==0 else "_nolegend_"
        if plot and rmse<10:
            #print(past.shape)
            plt.figure(figsize=(15, 3))
            plt.clf()
            plt.plot(range(len(past)), past, "x--y", label=label1)
            plt.plot(range(len(past),len(true)+len(past)), true, label=label2)
            plt.plot(range(len(past), len(pred)+len(past)), pred, "x--y",label=label3, color = 'red')
            plt.legend(loc='best')
            plt.title("{in1}, Predictions v.s. true values, RMSE:{in2}".format(in1=prediction_dates[j][:10], in2=rmse))
            plt.savefig("data/{in1}/visualizations/sample_{in2}.png".format(in1 = init_params.dataset, in2=j))
            plt.show()
    print(sum(rmse_list), len(rmse_list))
    print( 'Average RMSE per sample: ', sum(rmse_list) / float(len(rmse_list)) )
    print( 'Average MAE per sample: ', sum(mae_list) / float(len(mae_list)) )

In [18]:
### DEEPAR ON Dhaka
plot_prediction(prediction.history.values, prediction.ground_truth.values, prediction.prediction.values, prediction.Date.values, train = False, plot =1)

D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarnin

D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed 

No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot

D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
No handles with labels found to put in legend.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed 

9660.593678431942 359
Average RMSE per sample:  26.909731694796495
Average MAE per sample:  23.770516523278964


D:\Applications\Anaconda\envs\lfi\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [19]:
get_rmse_by_days(prediction)

Day: 0 24.26485681470336
Day: 1 29.288780027043
Day: 2 31.244250185123484
Day: 3 33.18252445438775
Day: 4 35.11225276805491
Day: 5 36.8363926997891
Day: 6 38.928347537885244

Mean RMSE:  32.69391492671241
